# Demonstrating the dataset generation for an ampl model

This notebook shows the generation of the dataset that are subsequently used to train a neural network for optimal control.

In [7]:
### %pylab notebook
from pyquad.ekin import*
from tqdm import tqdm

In [8]:
ampl_mod_path = "ampl/bebop_model_new.mod"

num = 10

# INITIAL CONDITIONS IN WORLD COORDINATES
x0 = np.random.uniform(-5.0, 5.0, num)
y0 = np.random.uniform(-5.0, 5.0, num)
z0 = np.random.uniform(-1.0, 1.0, num)

vx0 = np.random.uniform(-0.5,0.5, num)
vy0 = np.random.uniform(-0.5,0.5, num)
vz0 = np.random.uniform(-0.5,0.5, num)

phi0   = np.random.uniform(-40, 40, num)*np.pi/180
theta0 = np.random.uniform(-40, 40, num)*np.pi/180
psi0   = np.random.uniform(-180, 180, num)*np.pi/180

p0     = np.random.uniform(-1., 1., num)
q0     = np.random.uniform(-1., 1., num)
r0     = np.random.uniform(-1., 1., num)

utau0  = np.random.uniform(0, 1, [num, 4])

Mx_ext = 0*np.random.uniform(-.04, .04, num)
My_ext = 0*np.random.uniform(-.04, .04, num)
Mz_ext = 0*np.random.uniform(-.01, .01, num)

def transform(x,y,z,vx,vy,vz, phi,theta,psi):
    Rx = np.array([[1, 0, 0], [0, np.cos(phi), -np.sin(phi)], [0, np.sin(phi), np.cos(phi)]])
    Ry = np.array([[np.cos(theta), 0, np.sin(theta)], [0, 1, 0], [-np.sin(theta), 0, np.cos(theta)]])
    Rz = np.array([[np.cos(psi), -np.sin(psi), 0], [np.sin(psi), np.cos(psi), 0], [0, 0, 1]])
    R = Rz@Ry@Rx
    dx, dy, dz = -R.T@[x, y, z]
    vx, vy, vz =  R.T@[vx,vy,vz]
    return dx, dy, dz, vx, vy, vz

# IN BODY COORDINATES
dx0, dy0, dz0, vx0, vy0, vz0 = np.vectorize(transform)(x0, y0, z0, vx0, vy0, vz0, phi0, theta0, psi0)

parameters = {
        "dx0"    : dx0,
        "dy0"    : dy0, 
        "dz0"    : dz0,
        "vx0"    : vx0,
        "vy0"    : vy0,
        "vz0"    : vz0,
        "phi0"   : phi0,
        "theta0" : theta0,
        "psi0"   : psi0,
        "p0"     : p0,
        "q0"     : q0,
        "r0"     : r0,
        "utau0"  : utau0,
        "Mx_ext": Mx_ext,
        "My_ext": My_ext,
        "Mz_ext": Mz_ext
}


# FINAL CONDITIONS
dxn = 0.
dyn = 0.
dzn = 0.

vxn = 0.
vyn = 0
vzn = 0.

phin = 0.
thetan = 0.
psin = 0.

pn = 0.
qn = 0.
rn = 0.

fixed_parameters = {
    "dxn"         : dxn,
    "dyn"         : dyn, 
    "dzn"         : dzn,
    "vxn"         : vxn,
    "vyn"         : vyn,
    "vzn"         : vzn,
    "phin"        : phin,
    "thetan"      : thetan,
    "psin"        : psin,
    "pn"          : pn,
    "qn"          : qn,
    "rn"          : rn,
    "epsilon"     : 1.,
    "n"           : 100,
    "omega_max"   : 9500,
    "omega_min"   : 5500
}

In [9]:
def run_ampl_model(param_dict, file_path):
    model = AMPLModel(ampl_mod_path)
    model.setParameterValues(param_dict)
    model.setParameterValues(fixed_parameters)
    model.solve()
    soln_vals = model.getSolutionValues()
    obj_vals = model.getObjectiveValues()
    # save solution to file
    np.savez(file_path, **{"Success": model.checkSolved(), "t": soln_vals['timegrid'], **soln_vals, **obj_vals, **param_dict, **fixed_parameters})

In [10]:
name = 'HOVER_TO_HOVER_NOMINAL'
datafolder = 'datasets/' + name
datafile = 'datasets/' + name + '.npz'

# make folder if it doesn't exist
if not os.path.exists(datafolder):
    os.makedirs(datafolder)

In [11]:
##### trajectories to solve
index_set = set(range(num)) -  {int(file.replace('.npz','')) for file in os.listdir(datafolder)}

# solve trajectories in parallel
njobs = 1
sol_lst = joblib.Parallel(njobs)(joblib.delayed(run_ampl_model)(
    {key: val[idx] for key,val in parameters.items()},
    datafolder + '/' + str(idx)
) for idx in tqdm(index_set))

print('Generated', num,'trajectories in', datafolder)

  0%|          | 0/10 [00:00<?, ?it/s]

*******************************************************************************
*                                                                             *
* Please make sure that the AMPL directory is in the system search path, or   *
* add it before instantiating the AMPL object with:                           *
*                                                                             *
*     from amplpy import AMPL, add_to_path                                    *
*     add_to_path(r"full path to the AMPL installation directory")            *
*     ampl = AMPL()                                                           *
*                                                                             *
* Or, if you are using amplpy.modules, please make sure that they are installed: *
*                                                                             *
*     # Install solver modules (e.g., HiGHS, CBC, Gurobi)                     *
*     $ python -m amplpy.modules inst

RuntimeError: AMPL could not be started. Message from process thread:
cannot execute ampl: Permission denied



**View trajectories**

In [ ]:
def transform_back(dx,dy,dz,vx,vy,vz,phi,theta,psi):
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(phi), -np.sin(phi)],
        [0, np.sin(phi), np.cos(phi)]
    ])
    Ry = np.array([
        [np.cos(theta), 0, np.sin(theta)],
        [0, 1, 0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])
    Rz = np.array([
        [np.cos(psi), -np.sin(psi), 0],
        [np.sin(psi), np.cos(psi), 0],
        [0, 0, 1]
    ])
    R = Rz@Ry@Rx
    x_new, y_new, z_new = -R@[dx, dy, dz]
    vx_new, vy_new, vz_new = R@[vx, vy, vz]
    return x_new, y_new, z_new, vx_new, vy_new, vz_new, phi, theta, psi

In [ ]:
%matplotlib notebook
a = np.load(datafile)
num = a['dx'].shape[0]
print(num)

t,dx,dy,dz,vx,vy,vz,phi,theta,psi,u = (a[key][0:1000] for key in 't dx dy dz vx vy vz phi theta psi u'.split(' '))

In [ ]:
from quadcopter_animation import animation
import importlib
importlib.reload(animation)

x,y,z,vx,vy,vz,phi,theta,psi = np.vectorize(transform_back)(dx,dy,dz,vx,vy,vz,phi,theta,psi)
# t,x,y,z,vx,vy,vz,phi,theta,psi,u = (a[key] for key in 't x y z vx vy vz phi theta psi u'.split(' '))

animation.animate(t, x, y, z, phi, theta, psi, u, multiple_trajectories=True, simultaneous=False, step=1, waypoints=[np.array([0,0,0])], colors=[(255,0,0)]*100)